In [1]:
# imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
from scipy.stats import chisquare
from wordcloud import WordCloud

import numpy as np
from sklearn.metrics.classification import classification_report, accuracy_score
from nltk.stem.lancaster import LancasterStemmer 
from nltk.stem import SnowballStemmer, WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import cross_val_predict
wordnet_lemmatizer = WordNetLemmatizer()
snowball_stemmer = SnowballStemmer('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
import nltk
from sklearn import svm, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lp = LabelPropagation()
knn = KNeighborsClassifier()
rdmforest = RandomForestClassifier()
dtree = DecisionTreeClassifier()
nb = GaussianNB()
lr = LogisticRegression()
svc = LinearSVC()
mlp=MLPClassifier()
vect = TfidfVectorizer()
et = ExtraTreeClassifier()

In [2]:
# create generic path using so that the code can run in both windows and linux systems
raw_data_path = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'data','raw'))
processed_data_path = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'data','processed'))
interim_data_path = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'data','interim'))
final_path = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'data','final'))
print(raw_data_path)
print(processed_data_path)
print(interim_data_path)

G:\Git Projects\twitter_product_sentiment\data\raw
G:\Git Projects\twitter_product_sentiment\data\processed
G:\Git Projects\twitter_product_sentiment\data\interim


In [3]:
prefix = 'train'
prefix1='test'

In [4]:
df_test_original = pd.read_csv(os.path.abspath(os.path.join(raw_data_path,'test.csv')))

In [5]:
test_df_list = ['test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer', 'test_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer', 'test_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer', 'test_rt_rm_res_rs_rlt3_df_tweet_clean_wordnet_stemmer', 'test_rt_rm_res_rs_rlt3df', 'test_rt_rm_res_rsdf','test_rt_rm_resdf','test_rt_rmdf','test_rtdf']
train_df_list = ['train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer', 'train_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer', 'train_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer', 'train_rt_rm_res_rs_rlt3_df_tweet_clean_wordnet_stemmer', 'train_rt_rm_res_rs_rlt3df', 'train_rt_rm_res_rsdf','train_rt_rm_resdf','train_rt_rmdf','train_rtdf']

In [6]:
def print_report(y_test, y_pred):
    print(classification_report(y_test, y_pred))
    print('ACCURRACY::', accuracy_score(y_pred, y_test))

In [13]:
def test_model_only_tweet(df_test_original,vect=vect,model=mlp,vectname='tfidf',modelname='mlp'):
    for i,j in zip(test_df_list,train_df_list):
        print('reading {}.csv and {}.csv....'.format(i,j))
        df_test = pd.read_csv(os.path.abspath(os.path.join(processed_data_path,i+'.csv')))
        df_train = pd.read_csv(os.path.abspath(os.path.join(processed_data_path,j+'.csv')))
        df_test.dropna(inplace=True)
        df_train.dropna(inplace=True)
        print('dropping unnamed column')
        df_test.drop(['Unnamed: 0'],axis=1,inplace=True)
        df_train.drop(['Unnamed: 0'],axis=1,inplace=True)
        print('dividing train data to x and y')
        y=df_train['sentiment']
        trainX=df_train.drop(['sentiment'],axis=1)
        testX = df_test
        print('doing train test split of training data')
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(trainX, y, train_size=0.8)
        X_train_temp = X_train
        X_train=X_train['tweet']
        X_test_temp = X_test
        X_test=X_test['tweet']
        print('doing fit trainsforrm for x_train')
        x_train_vect = vect.fit_transform(X_train)
        print('fitting model...')
        model.fit(x_train_vect,y_train)
        print('model fitting completed')
        print('predicting X_test from training data')
        y_pred1 = model.predict(vect.transform(X_test))
        print('prediction completed')
        print_report(y_test,y_pred1)
        df_test1 = df_test['tweet']
        print('predicting the test data')
        y_pred = model.predict(vect.transform(df_test1))
        print('prediction completed')
        df1=pd.DataFrame(df_test_original['tweet_id'])
        filename = i+'_'+j+'_'+modelname+'_'+vectname+'.csv'
        print('storing preciction as {}'.format(filename))
        df1.reset_index(inplace=True)
        y_pred=pd.DataFrame(y_pred)
        y_pred.reset_index(inplace=True)
        y_pred.columns = ['index','sentiment']
        y_pred2 = pd.merge(df1,y_pred,on='index',how='left')
        y_pred2.drop(['index'],inplace=True,axis=1)
        print(filename)
        y_pred2.to_csv(os.path.abspath(os.path.join(final_data_path,filename)))
        print("="*40)

In [8]:
def test_model_only_tweet_toarray(df_test_original,vect=vect,model=mlp,vectname='tfidf',modelname='nb'):
    for i,j in zip(test_df_list,train_df_list):
        print('reading {}.csv and {}.csv....'.format(i,j))
        df_test = pd.read_csv(os.path.abspath(os.path.join(processed_data_path,i+'.csv')))
        df_train = pd.read_csv(os.path.abspath(os.path.join(processed_data_path,j+'.csv')))
        df_test.dropna(inplace=True)
        df_train.dropna(inplace=True)
        print('dropping unnamed column')
        df_test.drop(['Unnamed: 0'],axis=1,inplace=True)
        df_train.drop(['Unnamed: 0'],axis=1,inplace=True)
        print('dividing train data to x and y')
        y=df_train['sentiment']
        trainX=df_train.drop(['sentiment'],axis=1)
        testX = df_test
        print('doing train test split of training data')
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(trainX, y, train_size=0.8)
        X_train_temp = X_train
        X_train=X_train['tweet']
        X_test_temp = X_test
        X_test=X_test['tweet']
        print('doing fit trainsforrm for x_train')
        x_train_vect = vect.fit_transform(X_train)
        x_train_vect = x_train_vect.toarray()
        print('fitting model...')
        model.fit(x_train_vect,y_train)
        print('model fitting completed')
        print('predicting X_test from training data')
        y_pred1 = model.predict(vect.transform(X_test))
        print('prediction completed')
        print_report(y_test,y_pred1)
        df_test1 = df_test['tweet']
        print('predicting the test data')
        y_pred = model.predict(vect.transform(df_test1).toarray())
        print('prediction completed')
        df1=pd.DataFrame(df_test_original['tweet_id'])
        filename = i+'_'+j+'_'+modelname+'_'+vectname+'.csv'
        print('storing preciction as {}'.format(filename))
        df1.reset_index(inplace=True)
        y_pred=pd.DataFrame(y_pred)
        y_pred.reset_index(inplace=True)
        y_pred.columns = ['index','sentiment']
        y_pred2 = pd.merge(df1,y_pred,on='index',how='left')
        y_pred2.drop(['index'],inplace=True,axis=1)
        y_pred2.to_csv(os.path.abspath(os.path.join(final_data_path,filename)))
        print("="*40)

In [9]:
def add_col(df,col,df1):
    df[col] = df1[col].values
    return df
def add_col_val(df,col,df1):
    df[col] =  df1[col].values
    return df

In [10]:
def test_model_multiple_cols(df_test_original,vect=vect,model=mlp,vectname='tfidf',modelname='lr'):
    for i,j in zip(test_df_list,train_df_list):
        print('reading {}.csv and {}.csv....'.format(i,j))
        df_test = pd.read_csv(os.path.abspath(os.path.join(processed_data_path,i+'.csv')))
        df_train = pd.read_csv(os.path.abspath(os.path.join(processed_data_path,j+'.csv')))
        df_test.dropna(inplace=True)
        df_train.dropna(inplace=True)
        print('dropping unnamed column')
        df_test.drop(['Unnamed: 0'],axis=1,inplace=True)
        df_train.drop(['Unnamed: 0'],axis=1,inplace=True)
        print('dividing train data to x and y')
        y=df_train['sentiment']
        trainX=df_train.drop(['sentiment'],axis=1)
        testX = df_test
        print('doing train test split of training data')
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(trainX, y, train_size=0.8)
        X_train_temp = X_train
        X_train=X_train['tweet']
        X_test_temp = X_test
        X_test=X_test['tweet']
        print('doing fit trainsforrm for x_train')
        x_train_vect = vect.fit_transform(X_train)
        test_1_df = pd.DataFrame(x_train_vect.toarray(), columns=vect.get_feature_names())
        for col in cols_to_add:
            test_1_df=add_col(test_1_df,col,X_train_temp)
        print('fitting model...')
        model.fit(test_1_df,y_train)
        print('model fitting completed')
        print('predicting X_test from training data')
        x_test_df = vect.transform(X_test)
        x_test_1_df = pd.DataFrame(x_test_df.toarray(), columns=vect.get_feature_names())
        for col in cols_to_add:
            x_test_1_df=add_col(x_test_1_df,col,X_test_temp)
        y_pred1 = model.predict(x_test_1_df)
        print(x_test_1_df.shape)
        print('prediction completed')
        print_report(y_test,y_pred1)
        df_test1 = df_test['tweet']
        print('predicting the test data')
        df_test1 = vect.transform(df_test1)
        test_1_df = pd.DataFrame(df_test1.toarray(),columns=vect.get_feature_names())
        print(test_1_df.shape ,' ',df_test.shape )
        test_1_df['charcount']=df_test['charcount']
        test_1_df['countwords']=df_test['countwords']
        test_1_df['@counts']=df_test['@counts']
        test_1_df['#counts']=df_test['#counts']
        test_1_df['Capscounts']=df_test['Capscounts']
        test_1_df['count_excl_quest_marks']=df_test['count_excl_quest_marks']
        test_1_df['count_special_chars'] = df_test['count_special_chars'] 
        #for col in cols_to_add:
        #    df_test1=add_col_val(df_test1,col,df_test)
        y_pred = model.predict(df_test1)
        print('prediction completed')
        df1=pd.DataFrame(df_test_original['tweet_id'])
        filename = i+'_'+j+'_'+modelname+'_'+vectname+'.csv'
        print('storing preciction as {}'.format(filename))
        df1.reset_index(inplace=True)
        y_pred=pd.DataFrame(y_pred)
        y_pred.reset_index(inplace=True)
        y_pred.columns = ['index','sentiment']
        y_pred2 = pd.merge(df1,y_pred,on='index',how='left')
        y_pred2.drop(['index'],inplace=True,axis=1)
        y_pred2.to_csv(os.path.abspath(os.path.join(final_data_path,filename)))
        print("="*40)

In [15]:
final_data_path =  os.path.abspath(os.path.join(final_path,'lr'))
test_model_only_tweet(df_test_original,vect=vect,model=lr,vectname='tfidf',modelname='lr')

reading test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv....
dropping unnamed column
dividing train data to x and y
doing train test split of training data
doing fit trainsforrm for x_train
fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.50      0.02      0.04        94
           1       0.67      0.87      0.75       857
           2       0.59      0.41      0.49       478
           3       0.00      0.00      0.00        26

   micro avg       0.65      0.65      0.65      1455
   macro avg       0.44      0.33      0.32      1455
weighted avg       0.62      0.65      0.61      1455

ACCURRACY:: 0.6481099656357389
predicting the test data
prediction completed
storing preciction as test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_lancas

doing fit trainsforrm for x_train
fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       1.00      0.05      0.09        85
           1       0.65      0.89      0.75       829
           2       0.60      0.36      0.45       511
           3       0.00      0.00      0.00        30

   micro avg       0.64      0.64      0.64      1455
   macro avg       0.56      0.33      0.32      1455
weighted avg       0.64      0.64      0.59      1455

ACCURRACY:: 0.6378006872852233
predicting the test data
prediction completed
storing preciction as test_rtdf_train_rtdf_lr_tfidf.csv
test_rtdf_train_rtdf_lr_tfidf.csv


In [16]:
final_data_path =  os.path.abspath(os.path.join(final_path,'rforest'))
test_model_only_tweet(df_test_original,vect=vect,model=rdmforest,vectname='tfidf',modelname='rdmforest')

reading test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv....
dropping unnamed column
dividing train data to x and y
doing train test split of training data
doing fit trainsforrm for x_train
fitting model...


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.57      0.26      0.36        81
           1       0.67      0.84      0.74       850
           2       0.61      0.42      0.49       503
           3       0.25      0.05      0.08        21

   micro avg       0.65      0.65      0.65      1455
   macro avg       0.52      0.39      0.42      1455
weighted avg       0.63      0.65      0.63      1455

ACCURRACY:: 0.6487972508591066
predicting the test data
prediction completed
storing preciction as test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_rdmforest_tfidf.csv
test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_rdmforest_tfidf.csv
reading test_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_porter_ste

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.57      0.28      0.37        76
           1       0.69      0.83      0.75       892
           2       0.55      0.41      0.47       468
           3       0.00      0.00      0.00        19

   micro avg       0.65      0.65      0.65      1455
   macro avg       0.45      0.38      0.40      1455
weighted avg       0.63      0.65      0.63      1455

ACCURRACY:: 0.6515463917525773
predicting the test data
prediction completed
storing preciction as test_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_rdmforest_tfidf.csv
test_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_rdmforest_tfidf.csv
reading test_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_sno

...

doing fit trainsforrm for x_train
fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.50      0.21      0.29        77
           1       0.68      0.86      0.76       861
           2       0.63      0.41      0.50       489
           3       0.00      0.00      0.00        28

   micro avg       0.66      0.66      0.66      1455
   macro avg       0.45      0.37      0.39      1455
weighted avg       0.64      0.66      0.63      1455

ACCURRACY:: 0.659106529209622
predicting the test data
prediction completed
storing preciction as test_rt_rm_resdf_train_rt_rm_resdf_rdmforest_tfidf.csv
test_rt_rm_resdf_train_rt_rm_resdf_rdmforest_tfidf.csv
reading test_rt_rmdf.csv and train_rt_rmdf.csv....
dropping unnamed column
dividing train data to x and y
doing train test split of training data
doing fit trainsforrm for x_train


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.50      0.16      0.25        73
           1       0.67      0.88      0.76       890
           2       0.59      0.32      0.42       464
           3       0.20      0.04      0.06        28

   micro avg       0.65      0.65      0.65      1455
   macro avg       0.49      0.35      0.37      1455
weighted avg       0.63      0.65      0.61      1455

ACCURRACY:: 0.650171821305842
predicting the test data
prediction completed
storing preciction as test_rt_rmdf_train_rt_rmdf_rdmforest_tfidf.csv
test_rt_rmdf_train_rt_rmdf_rdmforest_tfidf.csv
reading test_rtdf.csv and train_rtdf.csv....
dropping unnamed column
dividing train data to x and y
doing train test split of training data
doing fit trainsforrm for x_train


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.57      0.14      0.23        90
           1       0.66      0.86      0.75       871
           2       0.56      0.35      0.43       471
           3       0.00      0.00      0.00        23

   micro avg       0.64      0.64      0.64      1455
   macro avg       0.45      0.34      0.35      1455
weighted avg       0.61      0.64      0.60      1455

ACCURRACY:: 0.6364261168384879
predicting the test data
prediction completed
storing preciction as test_rtdf_train_rtdf_rdmforest_tfidf.csv
test_rtdf_train_rtdf_rdmforest_tfidf.csv


In [17]:
final_data_path =  os.path.abspath(os.path.join(final_path,'svc'))
test_model_only_tweet(df_test_original,vect=vect,model=svc,vectname='tfidf',modelname='svc')

reading test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv....
dropping unnamed column
dividing train data to x and y
doing train test split of training data
doing fit trainsforrm for x_train
fitting model...
model fitting completed
predicting X_test from training data


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


prediction completed
              precision    recall  f1-score   support

           0       0.40      0.21      0.28        98
           1       0.72      0.82      0.77       880
           2       0.58      0.51      0.54       453
           3       0.00      0.00      0.00        24

   micro avg       0.67      0.67      0.67      1455
   macro avg       0.42      0.39      0.40      1455
weighted avg       0.64      0.67      0.65      1455

ACCURRACY:: 0.6694158075601374
predicting the test data
prediction completed
storing preciction as test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_svc_tfidf.csv
test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_svc_tfidf.csv
reading test_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer.csv....
dropping unnamed column
dividing train data to x and y
doing

fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.52      0.27      0.36        89
           1       0.73      0.81      0.77       876
           2       0.60      0.53      0.56       473
           3       0.00      0.00      0.00        17

   micro avg       0.68      0.68      0.68      1455
   macro avg       0.46      0.40      0.42      1455
weighted avg       0.66      0.68      0.67      1455

ACCURRACY:: 0.6790378006872853
predicting the test data
prediction completed
storing preciction as test_rtdf_train_rtdf_svc_tfidf.csv
test_rtdf_train_rtdf_svc_tfidf.csv


In [18]:
final_data_path =  os.path.abspath(os.path.join(final_path,'mlp'))
test_model_only_tweet(df_test_original,vect=vect,model=mlp,vectname='tfidf',modelname='mlp')

reading test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer.csv....
dropping unnamed column
dividing train data to x and y
doing train test split of training data
doing fit trainsforrm for x_train
fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.32      0.28      0.30        87
           1       0.72      0.74      0.73       869
           2       0.54      0.56      0.55       464
           3       0.00      0.00      0.00        35

   micro avg       0.64      0.64      0.64      1455
   macro avg       0.39      0.39      0.39      1455
weighted avg       0.62      0.64      0.63      1455

ACCURRACY:: 0.6350515463917525
predicting the test data
prediction completed
storing preciction as test_rt_rm_res_rs_rlt3_df_tweet_clean_lancaster_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_lancas

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.39      0.35      0.37        85
           1       0.73      0.74      0.73       877
           2       0.56      0.56      0.56       477
           3       0.07      0.06      0.06        16

   micro avg       0.65      0.65      0.65      1455
   macro avg       0.44      0.43      0.43      1455
weighted avg       0.65      0.65      0.65      1455

ACCURRACY:: 0.6487972508591066
predicting the test data
prediction completed
storing preciction as test_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_mlp_tfidf.csv
test_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_porter_stemmer_mlp_tfidf.csv
reading test_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemme

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


fitting model...
model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.42      0.32      0.36        91
           1       0.70      0.74      0.72       845
           2       0.56      0.54      0.55       497
           3       0.06      0.05      0.05        22

   micro avg       0.64      0.64      0.64      1455
   macro avg       0.44      0.41      0.42      1455
weighted avg       0.63      0.64      0.63      1455

ACCURRACY:: 0.6357388316151202
predicting the test data
prediction completed
storing preciction as test_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer_mlp_tfidf.csv
test_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer_train_rt_rm_res_rs_rlt3_df_tweet_clean_snowball_stemmer_mlp_tfidf.csv
reading test_rt_rm_res_rs_rlt3_df_tweet_clean_wordnet_stemmer.csv and train_rt_rm_res_rs_rlt3_df_tweet_clean_wordnet_

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.53      0.35      0.42        89
           1       0.72      0.74      0.73       865
           2       0.55      0.56      0.55       479
           3       0.00      0.00      0.00        22

   micro avg       0.65      0.65      0.65      1455
   macro avg       0.45      0.41      0.43      1455
weighted avg       0.64      0.65      0.64      1455

ACCURRACY:: 0.6460481099656358
predicting the test data
prediction completed
storing preciction as test_rt_rmdf_train_rt_rmdf_mlp_tfidf.csv
test_rt_rmdf_train_rt_rmdf_mlp_tfidf.csv
reading test_rtdf.csv and train_rtdf.csv....
dropping unnamed column
dividing train data to x and y
doing train test split of training data
doing fit trainsforrm for x_train
fitting model...


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


model fitting completed
predicting X_test from training data
prediction completed
              precision    recall  f1-score   support

           0       0.52      0.31      0.39        90
           1       0.69      0.75      0.72       833
           2       0.55      0.52      0.54       508
           3       0.10      0.04      0.06        24

   micro avg       0.63      0.63      0.63      1455
   macro avg       0.46      0.41      0.43      1455
weighted avg       0.62      0.63      0.62      1455

ACCURRACY:: 0.6329896907216495
predicting the test data
prediction completed
storing preciction as test_rtdf_train_rtdf_mlp_tfidf.csv
test_rtdf_train_rtdf_mlp_tfidf.csv


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
test_model_only_tweet_toarray(df_test_original)

In [ ]:
df_test = pd.read_csv(os.path.abspath(os.path.join(processed_data_path,'test_rt_rm_res_rs_rlt3df.csv')))

In [ ]:
df_test.columns

In [ ]:
cols_to_add = ['charcount', 'countwords','@counts', '#counts', 'Capscounts', 'count_excl_quest_marks', 'count_special_chars']

In [ ]:
['', '','', '#', '', '',' ', '']

In [ ]:
test_model_only_tweet(df_test_original=df_test_original,model = et, modelname='et')

In [ ]:
test_model_only_tweet(df_test_original=df_test_original,model = knn, modelname='knn')

In [ ]:
test_model_only_tweet(df_test_original=df_test_original,model = lr, modelname='lr')